In [1]:
import pandas as pd
import polars 
import torch
import numpy as np
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
import math
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from models.datasets import *
from models.fcn import *

In [2]:
df = polars.read_csv('data/train.csv')
test_df = polars.read_csv('data/test.csv')

In [3]:
w = torch.load('data/weight.pt')
mask = w != 0

In [4]:
SRC_COLS = df.columns[1:557]
TGT_COLS = df.columns[557:]

for col in SRC_COLS:
    df = df.with_columns(polars.col(col).cast(polars.Float32))
    test_df = test_df.with_columns(polars.col(col).cast(polars.Float32))

for col in TGT_COLS:
    df = df.with_columns(polars.col(col).cast(polars.Float32))


src = torch.tensor(df.select(SRC_COLS).to_numpy(), dtype=torch.float32)
test_src = torch.tensor(test_df.select(SRC_COLS).to_numpy(), dtype=torch.float32)
label = torch.tensor(df.select(TGT_COLS).to_numpy(), dtype=torch.float32)
src_mu = src.mean(axis=0)
src_std = torch.maximum(src.std(axis=0), torch.tensor(1e-8))
label_mu = label.mean(axis=0)
label_std = torch.maximum(label.std(axis=0), torch.tensor(1e-8))
test_src = (test_src - src_mu) / src_std

In [5]:
input_seq_name = [
    "state_t", "state_q0001", "state_q0002", "state_q0003", "state_u", "state_v", "pbuf_ozone", "pbuf_CH4", "pbuf_N2O"
]
input_scl_name = [
    "state_ps", "pbuf_SOLIN", "pbuf_LHFLX", "pbuf_SHFLX", "pbuf_TAUX", "pbuf_TAUY", "pbuf_COSZRS", "cam_in_ALDIF", "cam_in_ALDIR", "cam_in_ASDIF", "cam_in_ASDIR", "cam_in_LWUP", "cam_in_ICEFRAC", "cam_in_LANDFRAC", "cam_in_OCNFRAC", "cam_in_SNOWHLAND"
]
input_seq_idx = [[idx - 1 for idx, column in enumerate(df.columns) if 
                    column.startswith(var)] for var in input_seq_name]
input_scl_idx = [[idx - 1 for idx, column in enumerate(df.columns) if 
                    column.startswith(var)] for var in input_scl_name]
test_seq = torch.stack([test_src[:, i] for i in input_seq_idx], dim=-1)
test_scl = torch.stack([test_src[:, i].repeat(1, 60) for i in input_scl_idx], dim=-1)
input_dim = test_seq.size(-1)
scalar_dim = test_scl.size(-1)

In [6]:
test_ds = MixTestDataset(test_seq, test_scl)
test_loader = DataLoader(test_ds, batch_size=1024, shuffle=False)

In [7]:
ckpt = "/m9400/users/lkv6309/leap/ckpt/fnc-epoch=332-val_loss=0.195.ckpt"
model = AttentionFCN.load_from_checkpoint(ckpt)
# model = TransformerFilteredModel.load_from_checkpoint(ckpt)
checkpoint_callback = ModelCheckpoint(
    dirpath='ckpt/',
    filename='te-base-{epoch:02d}-{val_loss:.2f}',
    save_top_k=-1,
    monitor='val_loss',
    mode='min'
)
logger = TensorBoardLogger(save_dir="logger")
trainer = Trainer(
    logger=logger,
    callbacks=[checkpoint_callback],
    max_epochs=10,
    accelerator="gpu",
)

Trainer will use only 1 of 8 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=8)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [9]:
pred = trainer.predict(model, test_loader)

You are using a CUDA device ('NVIDIA H100 80GB HBM3') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]
/m9400/users/lkv6309/miniconda3/envs/rise/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=111` in the `DataLoader` to improve performance.


Predicting: |          | 0/? [00:00<?, ?it/s]

In [ ]:
# ckpt = "/m9400/users/lkv6309/leap/ckpt/tf-epoch=111-val_loss=0.32.ckpt"
# model2 = TransformerFilteredModel.load_from_checkpoint(ckpt)
# # model = TransformerFilteredModel.load_from_checkpoint(ckpt)
# checkpoint_callback = ModelCheckpoint(
#     dirpath='ckpt/',
#     filename='te-base-{epoch:02d}-{val_loss:.2f}',
#     save_top_k=-1,
#     monitor='val_loss',
#     mode='min'
# )
# logger = TensorBoardLogger(save_dir="logger")
# trainer2 = Trainer(
#     logger=logger,
#     callbacks=[checkpoint_callback],
#     max_epochs=10,
#     accelerator="gpu",
# )
# pred2 = trainer2.predict(model2, test_loader)

In [10]:
preds = torch.cat(pred)
for i in range(label_std.shape[0]):
    if label_std[i] < 1.1e-8:
        preds[:,i] = 0
preds = (preds * label_std + label_mu)  * w
# preds2 = torch.cat(pred2)
# for i in range(label_std.shape[0]):
#     if label_std[i] < 1.1e-8:
#         preds2[:,i] = 0
# preds = ((preds+preds2)/2 * label_std + label_mu)  * w

In [11]:
print(preds[:,mask].std())


tensor(0.8005)


In [12]:
print(preds[0])

tensor([-3.9851e-01, -5.7869e-01, -3.9847e-01, -6.5252e-01, -9.1345e-01,
        -9.9535e-01, -9.7802e-01, -1.0455e+00, -1.0773e+00, -1.0485e+00,
        -9.9329e-01, -9.4116e-01, -9.2869e-01, -7.9476e-01, -3.1186e-01,
         2.8042e-01,  5.0008e-01,  1.0036e-01,  3.7329e-01, -2.0165e-01,
        -1.6992e-01, -3.8444e-01, -5.3666e-01, -5.8915e-01, -5.5097e-01,
        -5.3382e-01, -5.7886e-01, -6.1382e-01, -6.1019e-01, -6.0842e-01,
        -5.8934e-01, -5.1812e-01, -4.8346e-01, -4.1892e-01, -3.9040e-01,
        -4.0021e-01, -3.6307e-01, -3.6683e-01, -3.7326e-01, -3.6886e-01,
        -3.4299e-01, -3.3935e-01, -3.6308e-01, -4.6040e-01, -3.7759e-01,
        -4.2426e-01, -4.0262e-01, -5.1220e-01, -4.3263e-01, -4.4767e-01,
        -5.6420e-01, -7.8237e-01, -7.7378e-01, -8.2859e-01, -6.4627e-01,
        -5.2525e-01, -5.0269e-01, -5.2644e-01, -4.3785e-01, -3.1730e-01,
        -0.0000e+00, -0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0

In [13]:
print(0.24 * mask.sum() / torch.logical_and(mask, label_std > 1.1e-8).sum())
print(0.21 * len(mask) / torch.logical_and(mask, label_std > 1.1e-8).sum())

tensor(0.3716)
tensor(0.3923)


In [14]:
print(preds[0])

tensor([-3.9851e-01, -5.7869e-01, -3.9847e-01, -6.5252e-01, -9.1345e-01,
        -9.9535e-01, -9.7802e-01, -1.0455e+00, -1.0773e+00, -1.0485e+00,
        -9.9329e-01, -9.4116e-01, -9.2869e-01, -7.9476e-01, -3.1186e-01,
         2.8042e-01,  5.0008e-01,  1.0036e-01,  3.7329e-01, -2.0165e-01,
        -1.6992e-01, -3.8444e-01, -5.3666e-01, -5.8915e-01, -5.5097e-01,
        -5.3382e-01, -5.7886e-01, -6.1382e-01, -6.1019e-01, -6.0842e-01,
        -5.8934e-01, -5.1812e-01, -4.8346e-01, -4.1892e-01, -3.9040e-01,
        -4.0021e-01, -3.6307e-01, -3.6683e-01, -3.7326e-01, -3.6886e-01,
        -3.4299e-01, -3.3935e-01, -3.6308e-01, -4.6040e-01, -3.7759e-01,
        -4.2426e-01, -4.0262e-01, -5.1220e-01, -4.3263e-01, -4.4767e-01,
        -5.6420e-01, -7.8237e-01, -7.7378e-01, -8.2859e-01, -6.4627e-01,
        -5.2525e-01, -5.0269e-01, -5.2644e-01, -4.3785e-01, -3.1730e-01,
        -0.0000e+00, -0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0

In [15]:
ss = pd.read_csv("sample_submission.csv")
ss.iloc[:,1:] = preds.numpy()
test_polars = polars.from_pandas(ss[["sample_id"]+TGT_COLS])
test_polars.write_csv("submission.csv")

In [17]:
! kaggle competitions submit -c leap-atmospheric-physics-ai-climsim -f submission.csv -m "fcn val 0.20"

100%|███████████████████████████████████████| 3.73G/3.73G [00:34<00:00, 116MB/s]
Successfully submitted to LEAP - Atmospheric Physics using AI (ClimSim)